# Pedestrian Crash - Recurrent Autoregressive NN

## Read the dataset

I am reading the dataset from a file.
The dataset contains timeseries for the XYZ coordinate of the head of the pedestrian.
I will try to predict the next x coordinate, given the previous series.

In the previous section, we built a single-step baseline model that simply predicted the
last known value. For multi-step models, we’ll predict more than one timestep into
the future. In this case, we’ll forecast the traffic volume for the next 24 hours of data
given an input of 24 hours.
Again, the first step is to generate the appropriate window of data. Because we wish
to predict 24 timesteps into the future with an input of 24 hours, the input width is 24,
the label width is 24, and the shift is also 24.


AUTO DEN EXEI TELEIOSEI, NA VALO TO TEST SET GIA TRAINING< GIA NA DO TI THA GINEI. TORA EINAI POLU DUSKOLA NA EKPAIDEUTEI, THELEI POLLU ORA,
OPOTE OTAN EIMAI SIGOYROS, NA VALO TO MEGALO...

## Importing the libraries

In [1]:
from pathlib import Path
from src.dataset_reader import DatasetReaderCSV
from src.utilities import to_scrollable_table
from src.plot import plot_car_attributes_onehot, plot_pedestrian_attributes, plot_timeseries_random_entry
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import ast
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense



In [ ]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices("GPU")))

## Read the dataset

I am reading the dataset from a file.
I've previously exported the dataset using another script.

In [ ]:
train_path = Path("C:\\Users\geork\projects\AIThesis\src\datasets\head_coordinates_time_series\\train_set.csv")
test_path  = Path("C:\\Users\geork\projects\AIThesis\src\datasets\head_coordinates_time_series\\test_set.csv")
val_path   = Path("C:\\Users\geork\projects\AIThesis\src\datasets\head_coordinates_time_series\\validation_set.csv")


reader = DatasetReaderCSV(train_path)
reader.read()
train_df = reader.convert_to_dataframe()
train_df = train_df.drop(columns=["Position", "Path"], errors="ignore")

reader = DatasetReaderCSV(test_path)
reader.read()
test_df = reader.convert_to_dataframe()
test_df = test_df.drop(columns=["Position", "Path"], errors="ignore")

reader = DatasetReaderCSV(val_path)
reader.read()
val_df = reader.convert_to_dataframe()
val_df = val_df.drop(columns=["Position", "Path"], errors="ignore")


In [ ]:
print(train_df.describe)

## Dealing with NaN values

In [ ]:
total = train_df.isna().sum() + test_df.isna().sum() + val_df.isna().sum()
print(total)

In [ ]:
# Assuming val_df is already defined and contains the required data
fig, axes = plt.subplots(nrows=3, ncols=1, figsize=(13, 18))

# Plot Head_X_Coordinate
axes[0].plot(val_df['Head_X_Coordinate'])
axes[0].set_xlabel('Timestep')
axes[0].set_ylabel('Head_X_Coordinate')
axes[0].set_title('Head X Coordinate over Time')
axes[0].set_xlim(0, len(val_df['Head_X_Coordinate']))

# Plot Head_Y_Coordinate
axes[1].plot(val_df['Head_Y_Coordinate'])
axes[1].set_xlabel('Timestep')
axes[1].set_ylabel('Head_Y_Coordinate')
axes[1].set_title('Head Y Coordinate over Time')
axes[1].set_xlim(0, len(val_df['Head_Y_Coordinate']))

# Plot Head_Z_Coordinate
axes[2].plot(val_df['Head_Z_Coordinate'])
axes[2].set_xlabel('Timestep')
axes[2].set_ylabel('Head_Z_Coordinate')
axes[2].set_title('Head Z Coordinate over Time')
axes[2].set_xlim(0, len(val_df['Head_Z_Coordinate']))

# Adjust layout
fig.autofmt_xdate()
plt.tight_layout()

plt.show()

In [ ]:
from src.data_window import DataWindow
from src.autoregressive import AutoRegressive, compile_and_fit

#multi_window = DataWindow(input_width=200, label_width=101, shift=301, label_columns=["Head_X_Coordinate", "Head_Y_Coordinate", "Head_Z_Coordinate"])
multi_window = DataWindow(200, 101, 301, train_df, val_df, test_df, 
                          label_columns=["Head_X_Coordinate"])

AR_LSTM = AutoRegressive(units=32, out_steps=101, train_df=train_df)
history = compile_and_fit(AR_LSTM, multi_window, 3, 1)

# Assume `window` is an object containing your training and validation data
history = compile_and_fit(AR_LSTM, multi_window, checkpoint_path='C:\\Users\geork\projects\AIThesis\src\checkpoint_rnn_models')


ms_val_performance = {}
ms_performance = {}
ms_val_performance["AR - LSTM"] = AR_LSTM.evaluate(multi_window.val)
ms_performance["AR - LSTM"] = AR_LSTM.evaluate(multi_window.test, verbose=0)

multi_window.plot(AR_LSTM)